In [ ]:
from src.data.s3_utils import S3Handler

# Initialize the handler
s3_handler = S3Handler()

In [ ]:
def  upload_raw_data_to_s3(local_raw_data_dir, mailbox_name):
    import os
    from src.data.s3_utils import S3Handler

    # Initialize S3 handler
    s3_handler = S3Handler()

    # List existing buckets
    buckets = s3_handler.list_buckets()
    print("Existing buckets:", buckets)

    # Define your project bucket name
    project_bucket = "olkoa-projects"

    # Create the bucket if it doesn't exist
    if project_bucket not in buckets:
        s3_handler.create_bucket(project_bucket)
        print(f"Bucket '{project_bucket}' created.")
    else:
        print(f"Bucket '{project_bucket}' already exists.")

    print(s3_handler.list_directories(project_bucket, prefix=""))

    # Upload raw data directory to S3
    # local_raw_data_dir = "data/Projects/Projet Demo/Boîte mail de Céline/raw/"
    s3_prefix = f"{mailbox_name}/raw/"

    s3_handler.upload_directory(
        local_dir=local_raw_data_dir,
        bucket_name=project_bucket,
        s3_prefix=s3_prefix
    )
    print(f"Uploaded contents of '{local_raw_data_dir}' to 's3://{project_bucket}/{s3_prefix}'")

In [ ]:
# local_raw_data_dir = "/Users/julienrm/Documents/joel_zipped/"
# mailbox_name = "joel_zipped_full"

# upload_raw_data_to_s3(local_raw_data_dir=local_raw_data_dir, mailbox_name=mailbox_name)

Existing buckets: ['highco', 'olkoa-celine', 'olkoa-joel', 'olkoa-projects']
Bucket 'olkoa-projects' already exists.


INFO:src.data.s3_utils:Found 9 directories in olkoa-projects/


['celinepst', 'celinev3', 'dino', 'jo_3_eml', 'joel_sample_projet', 'pst500', 'pst500v2', 'single_eml', 'single_eml_proj']


INFO:src.data.s3_utils:File /Users/julienrm/Documents/joel_zipped/joel_mailbox.zip uploaded to olkoa-projects/joel_zipped_full/raw/joel_mailbox.zip


Uploaded /Users/julienrm/Documents/joel_zipped/joel_mailbox.zip to olkoa-projects/joel_zipped_full/raw/joel_mailbox.zip
Uploaded contents of '/Users/julienrm/Documents/joel_zipped/' to 's3://olkoa-projects/joel_zipped_full/raw/'


In [12]:
s3_handler.list_objects(bucket_name="olkoa-projects", prefix="joel_zipped_full/")

[{'key': 'joel_zipped_full/raw/joel_mailbox.zip',
  'size': 24241336594,
  'last_modified': datetime.datetime(2025, 9, 24, 15, 14, 54, tzinfo=tzutc())}]

In [14]:
# # small functions to list folder at a path
# local_raw_data_dir = "/Users/julienrm/Documents/joel_mailbox/"

# def list_local_folders(local_dir):
#     import os
#     folder_list = []
#     for root, dirs, files in os.walk(local_dir):
#         for dir in dirs:
#             folder_list.append(os.path.relpath(os.path.join(root, dir), local_dir))
#     return folder_list
# print(list_local_folders(local_raw_data_dir))

In [ ]:
# check if it is correctly uploaded
print(s3_handler.list_objects("olkoa-projects", prefix="joel_zipped_full/raw/"))


INFO:src.data.s3_utils:Found 0 directories in olkoa-projects/joel_zipped_full/raw/


[{'key': 'joel_zipped_full/raw/joel_mailbox.zip', 'size': 24241336594, 'last_modified': datetime.datetime(2025, 9, 24, 15, 14, 54, tzinfo=tzutc())}]
[]


In [18]:
# from src.data.s3_utils import S3Handler

# s3_handler = S3Handler()
# source_bucket = "olkoa-joel"                  # where the objects live now
# source_keys = [
#     obj["key"]
#     for obj in s3_handler.list_objects(bucket_name=source_bucket, prefix="")
#     if obj["key"] and not obj["key"].endswith("/")  # skip any directory markers
# ]

# dest_bucket = "olkoa-projects"                    # keep same bucket (adjust if different)
# dest_prefix = "joel/raw"
# for key in source_keys:
#     dest_key = f"{dest_prefix}/{key}"
#     s3_handler.copy_object(
#         source_bucket=source_bucket,
#         source_key=key,
#         dest_bucket=dest_bucket,
#         dest_key=dest_key,
#     )

In [21]:
keys = [
    obj["key"]
    for obj in s3_handler.list_objects(
        bucket_name="olkoa-projects", prefix="joel_full_mailbox/"
    )
    if obj["key"]
]
chunk_size = 1000  # S3 limit per DeleteObjects call
for i in range(0, len(keys), chunk_size):
    batch = keys[i:i + chunk_size]
    result = s3_handler.delete_objects(
        bucket_name="olkoa-projects",
        object_keys=batch,
    )
    print("Deleted:", result["Deleted"])
    if result["Errors"]:
        print("Errors:", result["Errors"])
if not keys:
    print("Nothing to delete.")

INFO:src.data.s3_utils:Deleted 1000 objects from olkoa-projects


Deleted: ['joel_full_mailbox/raw/.DS_Store', 'joel_full_mailbox/raw/.gitkeep', 'joel_full_mailbox/raw/Sent-6/1564671012313110001.eml', 'joel_full_mailbox/raw/Sent-6/1564672624447110001.eml', 'joel_full_mailbox/raw/Sent-6/1564673497321110001.eml', 'joel_full_mailbox/raw/Sent-6/1564673498315110002.eml', 'joel_full_mailbox/raw/Sent-6/1564673498652110003.eml', 'joel_full_mailbox/raw/Sent-6/1564673498868110004.eml', 'joel_full_mailbox/raw/Sent-6/1564673499772110005.eml', 'joel_full_mailbox/raw/Sent-6/1564673581190110001.eml', 'joel_full_mailbox/raw/Sent-6/1564673582416110002.eml', 'joel_full_mailbox/raw/Sent-6/1564673583054110003.eml', 'joel_full_mailbox/raw/Sent-6/1564673583574110004.eml', 'joel_full_mailbox/raw/Sent-6/1564674826750110001.eml', 'joel_full_mailbox/raw/Sent-6/1564674853933110001.eml', 'joel_full_mailbox/raw/Sent-6/1564675409176110001.eml', 'joel_full_mailbox/raw/Sent-6/1564675410459110002.eml', 'joel_full_mailbox/raw/Sent-6/1564675427266110001.eml', 'joel_full_mailbox/raw/Se

INFO:src.data.s3_utils:Deleted 936 objects from olkoa-projects


Deleted: ['joel_full_mailbox/raw/Sent-6/1571112848652110002.eml', 'joel_full_mailbox/raw/Sent-6/1571112849355110003.eml', 'joel_full_mailbox/raw/Sent-6/1571113143110110001.eml', 'joel_full_mailbox/raw/Sent-6/1571113143445110002.eml', 'joel_full_mailbox/raw/Sent-6/1571113652532110001.eml', 'joel_full_mailbox/raw/Sent-6/1571114509603110001.eml', 'joel_full_mailbox/raw/Sent-6/1571116621460110001.eml', 'joel_full_mailbox/raw/Sent-6/1571118917831110001.eml', 'joel_full_mailbox/raw/Sent-6/1571118918538110002.eml', 'joel_full_mailbox/raw/Sent-6/1571127274553110001.eml', 'joel_full_mailbox/raw/Sent-6/1571138757260110001.eml', 'joel_full_mailbox/raw/Sent-6/1571138757671110002.eml', 'joel_full_mailbox/raw/Sent-6/1571138855575110001.eml', 'joel_full_mailbox/raw/Sent-6/1571138978943110002.eml', 'joel_full_mailbox/raw/Sent-6/1571157374288110001.eml', 'joel_full_mailbox/raw/Sent-6/1571157603389110001.eml', 'joel_full_mailbox/raw/Sent-6/1571157617563110001.eml', 'joel_full_mailbox/raw/Sent-6/15712034

In [24]:

# List buckets
buckets = s3_handler.list_buckets()
print(f"Available buckets: {buckets}")

Available buckets: ['highco', 'olkoa-celine', 'olkoa-joel', 'olkoa-projects']


In [25]:
project_bucket = "olkoa-projects"

if not project_bucket in buckets:
    # Create a new bucket
    s3_handler.create_bucket(project_bucket)


In [ ]:
# # Upload the JSON as a file
# s3_handler.upload_directory(
#     local_dir="data/Projects/Dans la vallée haha/valley/raw",
#     bucket_name=project_bucket,
#     s3_prefix="single_eml/raw/"
# )

In [26]:
# List buckets
buckets = s3_handler.list_buckets()
print(f"Available buckets: {buckets}")

Available buckets: ['highco', 'olkoa-celine', 'olkoa-joel', 'olkoa-projects']


In [ ]:
s3_handler.list_directories(bucket_name="olkoa-joel", prefix="")

INFO:src.data.s3_utils:Found 55 directories in olkoa-joel/


['1743755186948115500_1',
 '1743755186948115500_10',
 '1743755186948115500_11',
 '1743755186948115500_12',
 '1743755186948115500_13',
 '1743755186948115500_14',
 '1743755186948115500_15',
 '1743755186948115500_16',
 '1743755186948115500_17',
 '1743755186948115500_18',
 '1743755186948115500_19',
 '1743755186948115500_2',
 '1743755186948115500_20',
 '1743755186948115500_21',
 '1743755186948115500_22',
 '1743755186948115500_23',
 '1743755186948115500_24',
 '1743755186948115500_25',
 '1743755186948115500_26',
 '1743755186948115500_27',
 '1743755186948115500_28',
 '1743755186948115500_29',
 '1743755186948115500_3',
 '1743755186948115500_30',
 '1743755186948115500_31',
 '1743755186948115500_32',
 '1743755186948115500_33',
 '1743755186948115500_34',
 '1743755186948115500_35',
 '1743755186948115500_36',
 '1743755186948115500_37',
 '1743755186948115500_38',
 '1743755186948115500_39',
 '1743755186948115500_4',
 '1743755186948115500_40',
 '1743755186948115500_41',
 '1743755186948115500_42',
 '174

In [16]:
s3_handler.list_objects(bucket_name=project_bucket)

[{'key': 'pst500v2/pst500v2/raw/enron.pst',
  'size': 13984768,
  'last_modified': datetime.datetime(2025, 9, 17, 11, 5, 41, tzinfo=tzutc())}]

In [26]:

# delete from s3
s3_handler.delete_object(bucket_name=project_bucket, object_key='pst500/pst500mailbox/raw/enron.pst')


INFO:src.data.s3_utils:Object olkoa-projects/pst500/pst500mailbox/raw/enron.pst deleted successfully


True

In [ ]:
# # Get the list of objects and extract just the keys
# objects = s3_handler.list_objects(bucket_name=project_bucket)
# object_keys = [obj['key'] for obj in objects]
# s3_handler.delete_objects(bucket_name=project_bucket, object_keys=object_keys)

In [30]:
s3_handler.list_objects(bucket_name="olkoa-projects", prefix="")

[{'key': 'dino/raw/dinosaur_emails.mbox',
  'size': 10599,
  'last_modified': datetime.datetime(2025, 9, 19, 13, 17, 45, tzinfo=tzutc())},
 {'key': 'pst500/raw/enron.pst',
  'size': 13984768,
  'last_modified': datetime.datetime(2025, 9, 19, 13, 34, 29, tzinfo=tzutc())},
 {'key': 'pst500v2/pst500v2/raw/enron.pst',
  'size': 13984768,
  'last_modified': datetime.datetime(2025, 9, 17, 11, 5, 41, tzinfo=tzutc())}]

## Check s3 joel mailbox

In [10]:
s3_handler.list_objects(bucket_name="olkoa-joel", prefix="")

[{'key': '1743755186948115500_1/Archives/2018/1532012493613110001.eml',
  'size': 48138,
  'last_modified': datetime.datetime(2025, 4, 11, 14, 16, 19, tzinfo=tzutc())},
 {'key': '1743755186948115500_1/Archives/2022/1643830307189110001.eml',
  'size': 1013158,
  'last_modified': datetime.datetime(2025, 4, 11, 14, 16, 18, tzinfo=tzutc())},
 {'key': '1743755186948115500_1/Archives/2022/1654165495768110001.eml',
  'size': 121628,
  'last_modified': datetime.datetime(2025, 4, 11, 14, 16, 16, tzinfo=tzutc())},
 {'key': '1743755186948115500_1/Archives/2022/1654165541631110001.eml',
  'size': 121518,
  'last_modified': datetime.datetime(2025, 4, 11, 14, 16, 19, tzinfo=tzutc())},
 {'key': '1743755186948115500_1/Archives/2022/1654165569835110001.eml',
  'size': 121588,
  'last_modified': datetime.datetime(2025, 4, 11, 14, 16, 18, tzinfo=tzutc())},
 {'key': '1743755186948115500_1/Archives/2022/1654165593875110001.eml',
  'size': 121564,
  'last_modified': datetime.datetime(2025, 4, 11, 14, 16, 16

In [11]:
len(_)

177914

In [12]:
celine_json = s3_handler.list_objects(bucket_name="olkoa-celine", prefix="")
print(len(celine_json))
print(celine_json[:5])

19327
[{'key': 'Archive/1.eml', 'size': 1216884, 'last_modified': datetime.datetime(2025, 5, 21, 9, 46, 24, tzinfo=tzutc())}, {'key': 'Archive/10.eml', 'size': 3812, 'last_modified': datetime.datetime(2025, 5, 21, 9, 46, 25, tzinfo=tzutc())}, {'key': 'Archive/2.eml', 'size': 23929, 'last_modified': datetime.datetime(2025, 5, 21, 9, 46, 25, tzinfo=tzutc())}, {'key': 'Archive/3.eml', 'size': 339717, 'last_modified': datetime.datetime(2025, 5, 21, 9, 46, 25, tzinfo=tzutc())}, {'key': 'Archive/4.eml', 'size': 144422, 'last_modified': datetime.datetime(2025, 5, 21, 9, 46, 26, tzinfo=tzutc())}]


In [13]:
qjson = s3_handler.list_objects(bucket_name=project_bucket, prefix="olkoa")
print(len(qjson))
print(qjson[:5])

1
[{'key': 'olkoa/brutasse/brut/raw/celine.guyon.pst', 'size': 3449193472, 'last_modified': datetime.datetime(2025, 9, 17, 8, 35, 10, tzinfo=tzutc())}]


In [ ]:
s3_handler.